## Final Submission Code

This notebook contains the code used to create top submission for Kaggle 3.

In [1]:
import numpy as np
import pandas as pd
#%pip install xgboost
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVR
import re

## Import the 3 datasets

In [2]:
aa_gauges = pd.read_csv("AA Raining 2014-2023.csv")
plant_flows = pd.read_csv("plant flows 2014-2021 summary.csv")
weather = pd.read_csv("NOAA Data 2014-2023.csv")

## Data Preprocessing

In [3]:
weather['DATE'] = pd.to_datetime(weather['DATE']).dt.strftime('%Y/%m/%d')
plant_flows['Collection Date/Time'] = pd.to_datetime(plant_flows['Collection Date/Time'],format = '%Y/%m/%d 0:00').dt.strftime('%Y/%m/%d')

In [4]:
# Improved function to remove leading zeroes from day and month in date strings but not from time
def remove_leading_zeroes(date_str):
    # Split the string into date and time
    # This regex finds parts of the date that are of the form '/0' and replaces them with '/'
    new_date_part = re.sub(r'(?<=/| )0(?=[1-9])', '', date_str)
    # Reassemble the date and time parts
    new_date_str = new_date_part #+ ' ' + time_part
    return new_date_str

# Apply the transformation to the 'Collection Date/Time' column
weather['DATE'] = weather['DATE'].apply(remove_leading_zeroes)
plant_flows['Collection Date/Time'] = plant_flows['Collection Date/Time'].apply(remove_leading_zeroes)
weather.head()

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,PRCP,SN52,SNOW,SNWD,SX52,TMAX,TMIN,TOBS,WESD
0,USC00200230,"ANN ARBOR U OF MICH, MI US",42.29806,-83.66388,247.8,2014/1/1,0.33,30.0,4.7,6.0,31.0,16,12.0,12,NaN
1,USC00200230,"ANN ARBOR U OF MICH, MI US",42.29806,-83.66388,247.8,2014/1/2,0.41,30.0,4.3,8.0,30.0,15,10.0,10,NaN
2,USC00200230,"ANN ARBOR U OF MICH, MI US",42.29806,-83.66388,247.8,2014/1/3,0.01,30.0,0.2,7.0,30.0,16,-6.0,7,NaN
3,USC00200230,"ANN ARBOR U OF MICH, MI US",42.29806,-83.66388,247.8,2014/1/4,0.00,30.0,0.0,6.0,30.0,30,7.0,27,NaN
4,USC00200230,"ANN ARBOR U OF MICH, MI US",42.29806,-83.66388,247.8,2014/1/5,0.56,30.0,5.6,12.0,30.0,31,25.0,25,NaN


In [5]:
merged_data = pd.merge(aa_gauges, weather, left_on='period', right_on='DATE', how='outer')
merged_data.tail(100)

,period,Barton Pond,CityHall,Jackson,N Campus Pump Station,S Industrial,STATION,NAME,LATITUDE,LONGITUDE,...,DATE,PRCP,SN52,SNOW,SNWD,SX52,TMAX,TMIN,TOBS,WESD
3478,2023/7/11,0.21,0.38,0.26,NaN,0.22,USC00200230,"ANN ARBOR U OF MICH, MI US",42.29806,-83.66388,...,2023/7/11,0.51,67.0,0.0,0.0,70.0,86.0,57.0,74.0,NaN
3479,2023/7/12,0.41,0.09,0.34,NaN,0.38,USC00200230,"ANN ARBOR U OF MICH, MI US",42.29806,-83.66388,...,2023/7/12,0.18,67.0,0.0,0.0,69.0,78.0,62.0,69.0,NaN
3480,2023/7/13,0.13,0.03,0.08,NaN,0.17,USC00200230,"ANN ARBOR U OF MICH, MI US",42.29806,-83.66388,...,2023/7/13,0.32,67.0,0.0,0.0,69.0,77.0,59.0,74.0,NaN
3481,2023/7/14,0.01,0.05,0.07,NaN,0.10,USC00200230,"ANN ARBOR U OF MICH, MI US",42.29806,-83.66388,...,2023/7/14,0.03,66.0,0.0,0.0,69.0,84.0,55.0,79.0,NaN
3482,2023/7/15,0.45,0.61,0.46,NaN,0.74,USC00200230,"ANN ARBOR U OF MICH, MI US",42.29806,-83.66388,...,2023/7/15,0.51,68.0,0.0,0.0,69.0,79.0,64.0,72.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3573,2023/10/14,0.31,0.41,0.37,NaN,0.00,USC00200230,"ANN ARBOR U OF MICH, MI US",42.29806,-83.66388,...,2023/10/14,0.45,NaN,0.0,0.0,NaN,53.0,46.0,50.0,NaN
3574,2023/10/15,0.00,0.01,0.01,NaN,0.00,USC00200230,"ANN ARBOR U OF MICH, MI US",42.29806,-83.66388,...,2023/10/15,0.00,NaN,0.0,0.0,NaN,53.0,40.0,51.0,NaN
3575,2023/10/16,0.00,0.00,0.00,NaN,0.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3576,2023/10/17,0.00,0.00,0.00,NaN,0.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
final_merged_data = pd.merge(merged_data, plant_flows, left_on='DATE', right_on='Collection Date/Time', how='outer')

### Feature selection

In [7]:
df = final_merged_data

# Remove specific columns
df = df.drop(['STATION', 'NAME', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'DATE', 'Collection Date/Time'], axis=1)

# Convert 'period' to datetime
df['period'] = pd.to_datetime(df['period'])

# Filter the DataFrame
df = df[df['period'] <= pd.to_datetime('2023-02-02')]

df.tail(100)

,period,Barton Pond,CityHall,Jackson,N Campus Pump Station,S Industrial,PRCP,SN52,SNOW,SNWD,SX52,TMAX,TMIN,TOBS,WESD,Delivered water
3237,2022-10-26,0.20,NaN,0.18,0.2,0.16,0.21,49.0,0.0,0.0,49.0,68.0,45.0,45.0,NaN,NaN
3238,2022-10-27,0.00,NaN,0.00,0.0,0.00,0.00,45.0,0.0,0.0,49.0,55.0,29.0,49.0,NaN,NaN
3239,2022-10-28,0.00,NaN,0.00,0.0,0.00,0.00,44.0,0.0,0.0,46.0,60.0,29.0,55.0,NaN,NaN
3240,2022-10-29,0.00,NaN,0.00,0.0,0.00,0.00,43.0,0.0,0.0,45.0,64.0,28.0,59.0,NaN,NaN
3241,2022-10-30,0.00,NaN,0.00,0.0,0.00,0.00,43.0,0.0,0.0,45.0,59.0,30.0,57.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3332,2023-01-29,0.05,0.0,0.01,NaN,0.04,0.09,30.0,0.6,6.0,31.0,31.0,27.0,28.0,NaN,NaN
3333,2023-01-30,0.00,0.0,0.00,NaN,0.00,0.03,30.0,0.4,6.0,30.0,28.0,19.0,19.0,0.8,NaN
3334,2023-01-31,0.00,0.0,0.00,NaN,0.00,0.00,29.0,0.0,5.0,30.0,20.0,-3.0,14.0,NaN,NaN
3335,2023-02-01,0.00,0.0,0.00,NaN,0.00,0.00,29.0,0.0,5.0,29.0,26.0,-1.0,18.0,NaN,NaN


In [8]:
df.isnull().sum()

period                      0
Barton Pond              3013
CityHall                   98
Jackson                   195
N Campus Pump Station     109
S Industrial              366
PRCP                        0
SN52                        0
SNOW                      245
SNWD                      245
SX52                        0
TMAX                        0
TMIN                        0
TOBS                        0
WESD                     3243
Delivered water           399
dtype: int64

In [9]:
rows_with_null = df[df['Delivered water'].isnull()]
rows_with_null

,period,Barton Pond,CityHall,Jackson,N Campus Pump Station,S Industrial,PRCP,SN52,SNOW,SNWD,SX52,TMAX,TMIN,TOBS,WESD,Delivered water
344,2014-12-06,NaN,0.00,0.00,0.00,0.00,0.00,34.0,0.0,0.0,35.0,39.0,31.0,31.0,NaN,NaN
2939,2022-01-01,NaN,0.06,0.02,0.02,0.09,0.12,33.0,0.6,1.0,33.0,41.0,29.0,29.0,NaN,NaN
2940,2022-01-02,NaN,0.00,0.00,0.00,0.02,0.21,32.0,2.2,3.0,33.0,29.0,17.0,17.0,NaN,NaN
2941,2022-01-03,NaN,0.00,0.01,0.00,0.00,0.00,30.0,0.0,2.0,32.0,28.0,1.0,21.0,0.2,NaN
2942,2022-01-04,NaN,0.00,0.00,0.00,0.00,0.00,30.0,0.0,2.0,30.0,33.0,15.0,30.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3332,2023-01-29,0.05,0.00,0.01,NaN,0.04,0.09,30.0,0.6,6.0,31.0,31.0,27.0,28.0,NaN,NaN
3333,2023-01-30,0.00,0.00,0.00,NaN,0.00,0.03,30.0,0.4,6.0,30.0,28.0,19.0,19.0,0.8,NaN
3334,2023-01-31,0.00,0.00,0.00,NaN,0.00,0.00,29.0,0.0,5.0,30.0,20.0,-3.0,14.0,NaN,NaN
3335,2023-02-01,0.00,0.00,0.00,NaN,0.00,0.00,29.0,0.0,5.0,29.0,26.0,-1.0,18.0,NaN,NaN


In [10]:
# Impute the missing value for the one day in the training set period of time
df.at[344, 'Delivered water'] = 13.5

In [11]:
# Drop columns that are missing a majority of the values
columns_to_drop = ['WESD', 'Barton Pond']
df = df.drop(columns=columns_to_drop)

In [12]:
# Creating the cosine and sine of the month
df['month'] = df['period'].dt.month

df['sin_month'] = np.sin(2 * np.pi * df['month'] / 12)

df['cos_month'] = np.cos(2 * np.pi * df['month'] / 12)

columns_to_drop = ['month']
df = df.drop(columns=columns_to_drop)

### Imputation

In [13]:
# Imputing the missing values from selected columns 
imputer = IterativeImputer(max_iter=10, random_state=0, imputation_order='ascending')

# List of columns to impute
columns_to_impute = ['CityHall', 'Jackson', 'N Campus Pump Station', 'S Industrial', 'PRCP', 'SN52', 'SNOW', 'SNWD', 'SX52', 'TMAX', 'TMIN', 'TOBS']

# Fit and transform the specified columns
df[columns_to_impute] = imputer.fit_transform(df[columns_to_impute])

In [14]:
df.isnull().sum()

period                     0
CityHall                   0
Jackson                    0
N Campus Pump Station      0
S Industrial               0
PRCP                       0
SN52                       0
SNOW                       0
SNWD                       0
SX52                       0
TMAX                       0
TMIN                       0
TOBS                       0
Delivered water          398
sin_month                  0
cos_month                  0
dtype: int64

In [15]:
# Creating the train and test sets
train_data = df[df['period'] < '2022-01-01']
test_data = df[(df['period'] >= '2022-01-01') & (df['period'] <= '2023-02-02')]

X_train = train_data.drop(columns=['Delivered water', 'period'])
y_train = train_data['Delivered water']

X_test = test_data.drop(columns=['Delivered water', 'period'])
y_test = test_data['Delivered water']

## Modeling

In [16]:
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score


cv_splits = 5

xgb_model = XGBRegressor()

param_grid = {
    'n_estimators': [20, 50, 100, 150, 200],  
    'max_depth': [1, 3, 5, 7, 10],           
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
}

# Create a TimeSeriesSplit cross-validator
tscv = TimeSeriesSplit(n_splits=cv_splits)

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='neg_mean_squared_error',
                           cv=tscv, verbose=1, n_jobs=-1)

# Fit the GridSearchCV object to the data
grid_search.fit(X_train, y_train)

print("Best Hyperparameters:", grid_search.best_params_)
print("Best Mean Squared Error:", -grid_search.best_score_)

# Access the best model directly
best_xgb_model = grid_search.best_estimator_

cv_results = cross_val_score(best_xgb_model, X_train, y_train, scoring='neg_mean_squared_error', cv=tscv)
print("Cross-validated Mean Squared Error on the entire dataset:", np.mean(-cv_results))

Fitting 5 folds for each of 100 candidates, totalling 500 fits


Best Hyperparameters: {'learning_rate': 0.2, 'max_depth': 1, 'n_estimators': 100}
Best Mean Squared Error: 2.7208429599998207
Cross-validated Mean Squared Error on the entire dataset: 2.7208429599998207


In [65]:
param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [None, 1, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Create the RandomForestRegressor
rf_model = RandomForestRegressor()


cv_splits = 5 

# Create a TimeSeriesSplit cross-validator
tscv = TimeSeriesSplit(n_splits=cv_splits)

# Create the GridSearchCV object
grid_search_rf = GridSearchCV(estimator=rf_model, param_grid=param_grid, scoring='neg_mean_squared_error',
                               cv=tscv, verbose=1, n_jobs=-1)

# Fit the GridSearchCV object to the data
grid_search_rf.fit(X_train, y_train)

print("Best Hyperparameters:", grid_search_rf.best_params_)
print("Best Mean Squared Error:", -grid_search_rf.best_score_)

Fitting 5 folds for each of 540 candidates, totalling 2700 fits


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
900 fits failed out of a total of 2700.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
309 fits failed with the following error:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_

Best Hyperparameters: {'max_depth': 5, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 150}
Best Mean Squared Error: 2.7140041092534775


In [144]:
base_models = [
    ('rf', RandomForestRegressor(max_depth=5, max_features='log2', min_samples_leaf=1, min_samples_split=5, n_estimators=150)),
    ('xgb', xgb.XGBRegressor(learning_rate=0.2, n_estimators=100, max_depth=1))
]

meta_estimator = LinearRegression() 

# Create the stacking ensemble model
stacking_model = StackingRegressor(estimators=base_models, final_estimator=meta_estimator)

In [145]:
cv_splits = 5 

# Create a TimeSeriesSplit cross-validator
tscv = TimeSeriesSplit(n_splits=cv_splits)

# Perform time series cross-validation and calculate MSE for each fold
mse_scores = -cross_val_score(stacking_model, X_train, y_train, scoring='neg_mean_squared_error', cv=tscv)

for i, mse in enumerate(mse_scores, 1):
    print(f'Fold {i}: Mean Squared Error = {mse}')

print(f'Average Mean Squared Error: {np.mean(mse_scores)}')

Fold 1: Mean Squared Error = 2.9405026349374728
Fold 2: Mean Squared Error = 2.5677263944037487
Fold 3: Mean Squared Error = 1.8425716662095766
Fold 4: Mean Squared Error = 2.7974903213227704
Fold 5: Mean Squared Error = 3.552996851387093
Average Mean Squared Error: 2.740257573652132


In [146]:
# Train the stacking model on the training data of the best fold
stacking_model.fit(X_train, y_train)

# Make predictions on the test data of the best fold
predictions = stacking_model.predict(X_test)

In [147]:
predictions

array([11.39813084, 11.48120006, 11.6419695 , 11.4659974 , 11.58023427,
       11.58479329, 11.82683869, 11.85086753, 11.82528817, 11.86310039,
       11.85199283, 11.74214171, 11.75062066, 11.86275066, 11.86192382,
       11.86215399, 11.67738969, 11.67117862, 11.860854  , 11.86503545,
       11.77718776, 11.78532146, 11.54195166, 11.50778812, 11.77903407,
       11.78036768, 11.76896913, 11.78762996, 11.70433717, 11.78653554,
       11.76935945, 11.96815758, 11.8646735 , 11.94644886, 12.12812374,
       12.1266717 , 12.1161762 , 12.12569512, 12.12647669, 12.02897718,
       12.10710687, 11.88015893, 12.11914309, 12.14996594, 12.15372242,
       12.15521573, 12.04136702, 11.8643133 , 11.96992546, 12.06441277,
       12.03109189, 12.04418747, 11.88609369, 12.14886673, 12.14022838,
       11.96405788, 12.04194874, 12.03282936, 12.04135427, 12.03039419,
       12.01420234, 12.10567549, 12.03038419, 12.11113239, 12.10957635,
       11.90569695, 12.11754559, 12.11658349, 12.11581565, 11.98

## Creating the submission file

In [148]:
sample_submission = pd.read_csv('sample submission.csv')
sample_submission['Delivered water'] = predictions
sample_submission.to_csv('submission_5_tv.csv', index=False)